In [ ]:
# Loading modules
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# RFM ANALYSIS

In [2]:
def rfm():
    # Ensure that all columns are shown 
    pd.set_option('display.max_columns', None) 
    df = pd.read_excel("../input/turkish-market-sales-dataset-with-9000items/MarketSales.xlsx")
    df.head()

    # Returns the individual number of dates
    df["STARTDATE"].nunique()

    # Returns missing values
    df.isnull().sum()

    # query of data size
    df.shape

    # number of unexpected products
    df.CATEGORY_NAME1.nunique()
    df.CATEGORY_NAME2.nunique()
    df.CATEGORY_NAME3.nunique()

    # There are 354 products without any esthetics(beauty products).
    df.BRAND.nunique()

    # shows how many times the products have gone
    df.CATEGORY_NAME3.value_counts().head(30)

    # Which product is the most siparised?
    # negative value due to cancellations
    df.groupby("CATEGORY_NAME3") .agg({"AMOUNT":"sum"}).head()

    # Which product was purchased the most.
    df.groupby("CATEGORY_NAME3").agg({"AMOUNT":"sum"}).sort_values("AMOUNT" ,ascending = False)

    # Total Price
    df["TotalPrice"] = df["AMOUNT"] * df["PRICE"]
    df.head()

    # According to the invoice, the most likely 
    df.groupby("FICHENO").agg({"TotalPrice":"sum"}).head()

    # the city with the most shoppers
    df.CITY.value_counts()[0:5]

    # The city that spends the most money
    df.groupby("CITY").agg({"TotalPrice":"sum"}).sort_values("TotalPrice" , ascending = False).head()

    # Show quarters
    df.describe([0.01,0.05,0.1,0.25,0.5,0.75,0.9,0.95,0.99]).T

    # It's a show of outliers.
    for feature in ["AMOUNT","PRICE","TotalPrice"]:

        Q1 = df[feature].quantile(0.01)
        Q3 = df[feature].quantile(0.99)
        IQR = Q3-Q1
        upper = Q3 + 1.5*IQR
        lower = Q1 - 1.5*IQR

        if df[(df[feature] > upper) | (df[feature] < lower)].any(axis=None):
            print(feature,"yes")
            print(df[(df[feature] > upper) | (df[feature] < lower)].shape[0])
        else:
            print(feature, "no")


    # Recency - Recent date
    # Last purchased date.
    df["STARTDATE"].max()

    # I've added the last date of the day as the date of the day
    import datetime as dt
    today_date = dt.datetime(2017 , 11 ,16)
    today_date

    # We're looking at the last time when the clients shop.
    df.groupby("CLIENTCODE").agg({"STARTDATE":"max"}).head()

    # We're retiring from the last time the customers shopd.
    temp_df = (today_date - df.groupby("CLIENTCODE").agg({"STARTDATE":"max"}))

    # changing name 
    temp_df.rename(columns = {"STARTDATE":"Recency"} , inplace= True)

    # We only got the number of days.
    recency_df = temp_df.Recency.apply(lambda x : x.days)

    # Frequency (sıklık)
    freq_df = df.groupby("CLIENTCODE").agg({"FICHENO":"nunique"})

    # changing name
    freq_df.rename(columns = {"FICHENO" : "Frequency"} , inplace = True)

    # Monetary (the total amount of money left by the client)
    monetary_df = df.groupby("CLIENTCODE").agg({"TotalPrice":"sum"})
    monetary_df.head()

    # Changing name
    monetary_df.rename(columns= {"TotalPrice":"Monetary"} , inplace = True)

    # merge the dfs
    rfm = pd.concat([recency_df , freq_df , monetary_df] , axis = 1)

    # Scoring process
    rfm["RecencyScore"] = pd.qcut(rfm["Recency"], 5, labels = [5, 4 , 3, 2, 1])
    rfm.head()
    rfm["Recency"].min()
    rfm["Frequency"].min()

    # Scoring process
    rfm["FrequencyScore"]= pd.qcut(rfm["Frequency"].rank(method="first"),5, labels=[1,2,3,4,5])

    # Scoring process
    rfm["MonetaryScore"] = pd.qcut(rfm["Monetary"] , 5 , labels = [1,2,3,4,5])

    rfm = rfm.dropna()

    # RFM scores are equipped with categorical value
    rfm["rfm_score"] = (rfm.RecencyScore.astype(str)+
                        rfm.FrequencyScore.astype(str) +
                        rfm.MonetaryScore.astype(str))

    # Bringing in the people who place the most orders and are active on the site.
    rfm.loc[rfm.rfm_score=="555"]

    # Regular Expressions; RFM map was mapped using
    seg_map = {
        r'[1-2][1-2]': 'Hibernating',
        r'[1-2][3-4]': 'At Risk',
        r'[1-2]5': 'Can\'t Loose',
        r'3[1-2]': 'About to Sleep',
        r'33': 'Need Attention',
        r'[3-4][4-5]': 'Loyal Customers',
        r'41': 'Promising',
        r'51': 'New Customers',
        r'[4-5][2-3]': 'Potential Loyalists',
        r'5[4-5]': 'Champions'
    }

    # Monetary value is excluded ,  FrequencyScore vs RecencyScore degerlerinin birlestirilerek segment isminde degisken olusturulması.
    rfm["segment"] =rfm.RecencyScore.astype(str) + rfm.FrequencyScore.astype(str)
    rfm.head() 

    # I named it using regex by applying it to the all segment..
    rfm.segment = rfm.segment.replace(seg_map , regex = True)

    # df and rfm
    result = pd.merge(df, rfm, on='CLIENTCODE')
    
    # Median value of total expenditure of groups according to segment.
    # result.groupby("segment").agg({"TotalPrice":np.median})

    # Recency of segments , Average and media scrutiny according to frequency and monetary values.
    #rfm[["segment","Recency","Frequency","Monetary"]].groupby("segment").agg(["mean","median","count"])
    return result

In [ ]:
rfm()

# Bivariate Analysis

In [ ]:
# Bivariate Analysis on all columns

def normal( support , threshold):
    """
    support = 'min_support' value used in bivariate analysis
    threshold = The value of the "association_rules" function used in Bivariate analysis.
    """
    # loading libraries
    import matplotlib.pyplot as plt
    import pandas as pd
    import seaborn as sns
   
    # Identification of data using function
    def satislar_load():
        dff =pd.read_excel("../input/turkish-market-sales-dataset-with-9000items/MarketSales.xlsx")
        return dff
    
    # Assigning a data set
    df = satislar_load()
    
    # Selecting one of the cities in the data
    df = df.loc[df.BRANCH == "İstanbul Subesi"]


    # Dividing the date part of the data into days, months, years
    df["STARTDATE"] = df["STARTDATE"].astype('datetime64[ns]')
    df["year"] =df["STARTDATE"].dt.year
    df["month"] =df["STARTDATE"].dt.month
    df["day"] =df["STARTDATE"].dt.day
    df["day_name"]= df.STARTDATE.dt.day_name()

    # backup of df
    df_yedek = df.copy()

    # Taking the individual names of the products
    df_urunler = df_yedek.CATEGORY_NAME3.unique()

    # sale of products according to all times
    # for a in df_urunler:
        #for i in ["day"  ,"day_name"]:
            #sns.countplot(df_yedek.loc[df_yedek.CATEGORY_NAME3 == a, "CATEGORY_NAME3"], hue=df_yedek[i])
            #plt.show()


    ### Bivariate Analysis
    df_genel = df_yedek.copy()
     
    # Separation of products
    df_yedek = df_yedek.CATEGORY_NAME3.str.strip(",")
    
    # Data merge
    dff = pd.concat([df_genel.FICHENO , df_yedek] , axis = 1 )
    
    # Evaluation of missing value 
    dff = dff.dropna()

    # Combination of products taken together on each issued invoice with a single line by ,
    dff = dff.groupby('FICHENO')['CATEGORY_NAME3'].agg(','.join).reset_index()
    
    # Dropping FICHENO 
    dff = dff.drop("FICHENO" , axis = 1)
    
    # Separating products
    data = list(dff['CATEGORY_NAME3'].apply(lambda x:x.split(",")))
    

    # Encoding processing
    from mlxtend.preprocessing import TransactionEncoder
    tencoder = TransactionEncoder()
    te_data = tencoder.fit(data).transform(data)
    df = pd.DataFrame(te_data, columns=tencoder.columns_)
   

    # Using the Apriori algorithm - uses frequent itemsets to generate association
    # used for working on trasactional datasets
    # with association rule it determines how strongly or weakly 2 objects are connected
    from mlxtend.frequent_patterns import apriori,  association_rules
    df1 = apriori(df, min_support=support, use_colnames=True)
   
    # Scrapping the rules of bivariate
    df_association = association_rules(df1, metric = 'confidence', min_threshold =threshold)
    
    return df_association.sort_values(by='confidence', ascending=False).reset_index()



In [ ]:
normal( 0.01 , 0.5)

**EVALUATION IN DAYS**

In [ ]:
# EVALUATION IN DAYS
def day( support , threshold , day , plot = False ):
    """
    support = 'min_support' value used in bivariate analysis
    threshold = The value of the "association_rules" function used in Bivariate analysis.
    day = Applies bivariate analysis on a day-to-day basis to be selected
    plot = Returns gun-based graphics of products when "True" is made.
    """
    # Loading Libraries
    import matplotlib.pyplot as plt
    import pandas as pd
    import seaborn as sns
    
    # Identification of data to the function
    def satislar_load():
        dff =pd.read_excel("../input/turkish-market-sales-dataset-with-9000items/MarketSales.xlsx")
        return dff
    
    # Assigning a dataset
    df = satislar_load()
    
    # Selecting one of the cities in the data
    df = df.loc[df.BRANCH == "İstanbul Subesi"]


    ## ucuncu. adım

    # Dividing the date part of the data into days, months, years
    df["STARTDATE"] = df["STARTDATE"].astype('datetime64[ns]')
    df["year"] =df["STARTDATE"].dt.year
    df["month"] =df["STARTDATE"].dt.month
    df["day"] =df["STARTDATE"].dt.day
    df["day_name"]= df.STARTDATE.dt.day_name()


    # backup of df  
    df_yedek = df.copy()
    
    # days selected  
    df_yedek = df_yedek.loc[df_yedek.day_name == day]
    
    # Taking the individual names of the products
    df_urunler = df_yedek.CATEGORY_NAME3.unique()

    # sale of products according to all times
    
    if plot:
        for a in df_urunler:
            for i in ["hour" ,"day_name"]:
                sns.countplot(df_yedek.loc[df_yedek.CATEGORY_NAME3 == a, "CATEGORY_NAME3"], hue=df_yedek[i])
                plt.show()

    ### Bivariate Analysis
    df_genel = df_yedek.copy()
    
    # Separation of products with ;
    df_yedek = df_yedek.CATEGORY_NAME3.str.strip(",")

    # Data merge
    dff = pd.concat([df_genel.FICHENO , df_yedek] , axis = 1 )
    
    # Evaluation of missing value
    dff = dff.dropna()

    # Combination of products taken together on each issued invoice with a single line 
    dff = dff.groupby('FICHENO')['CATEGORY_NAME3'].agg(','.join).reset_index()

    # Drop FICHENO 
    dff = dff.drop("FICHENO" , axis = 1)
    
    # separation of products
    data = list(dff['CATEGORY_NAME3'].apply(lambda x:x.split(",")))
    

    # Encoding processing
    from mlxtend.preprocessing import TransactionEncoder
    tencoder = TransactionEncoder()
    te_data = tencoder.fit(data).transform(data)
    df = pd.DataFrame(te_data, columns=tencoder.columns_)
   

    # Using the Apriori algorithm
    from mlxtend.frequent_patterns import apriori,  association_rules
    df1 = apriori(df, min_support=support, use_colnames=True)
   
    # Scrapping the rules of bivariate
    df_association = association_rules(df1, metric = 'confidence', min_threshold =threshold)
    
    return df_association.sort_values(by='confidence', ascending=False).reset_index()

In [ ]:
day( 0.01 , 0.50 , "Monday" )

In [ ]:
day( 0.01 , 0.50 , "Tuesday" )

In [ ]:
day( 0.01 , 0.50 , "Wednesday" )

In [ ]:
day( 0.01 , 0.50 , "Thursday" )

In [ ]:
day( 0.01 , 0.50 , "Friday" )

In [ ]:
day( 0.01 , 0.50 , "Saturday" )

In [ ]:
day( 0.01 , 0.50 , "Sunday" )

# CLTV

In [ ]:
# CLTV - Customer Lifetime Value
# Assess financial value of each customer
# how much money has customer already spent at your store ; loyalty of customer
#import modules
import pandas as pd # for dataframes
import matplotlib.pyplot as plt # for plotting graphs
import seaborn as sns # for plotting graphs
import datetime as dt
import numpy as np

In [ ]:
def cltv():
    # Identification of data
    def satislar_load():
        dff =pd.read_excel("../input/turkish-market-sales-dataset-with-9000items/MarketSales.xlsx")
        return dff
    # Assigning a data set
    df = satislar_load()
    df.head()

    # Calulate total purchase
    df['TotalPurchase'] = df['AMOUNT'] * df['PRICE']

    df=df.groupby('CLIENTCODE').agg({'STARTDATE': lambda date: (date.max() - date.min()).days,
                                            'FICHENO': lambda num: len(num),
                                            'AMOUNT': lambda quant: quant.sum(),
                                            'TotalPurchase': lambda price: price.sum()})

    # Change the name of columns
    df.columns=['num_days','num_transactions','num_units','spent_money']
    df.head()


    #1. Calculate Average Order Value
   
    # Average Order Value
    df['avg_order_value']=df['spent_money']/df['num_transactions']

    #2. Calculate Purchase Frequency

    purchase_frequency=sum(df['num_transactions'])/df.shape[0]

    #3. Calculate Repeat Rate and Churn Rate

    # Repeat Rate
    repeat_rate=df[df.num_transactions > 1].shape[0]/df.shape[0]

    #Churn Rate
    churn_rate=1-repeat_rate

    # Profit Margin
    df['profit_margin']=df['spent_money']*0.10

    # Customer Value
    df['CLV']=(df['avg_order_value']*purchase_frequency)/churn_rate

    # Customer Lifetime Value
    df['cust_lifetime_value']=df['CLV']*df['profit_margin']
    return df

In [ ]:
cltv()